# Simulations of _E. coli_

This notebook contains simple tests of the E. coli model `iML1515` both in regular and ecModel format.

Benjamín J. Sánchez, 2019-10-14

## 1. Loading models

* Metabolic model: https://github.com/SysBioChalmers/ecModels/blob/chore/updateiML1515/eciML1515/model/eciML1515.xml
* ecModel: https://github.com/SysBioChalmers/ecModels/blob/chore/updateiML1515/eciML1515/model/eciML1515.xml

(temporal, eventually they will be available in the master branch)

In [1]:
import cobra

! [ ! -f "iML1515.xml" ] && curl -O -L "https://raw.githubusercontent.com/SysBioChalmers/ecModels/chore/updateiML1515/eciML1515/model/iML1515.xml"
! [ ! -f "eciML1515.xml" ] && curl -O -L "https://raw.githubusercontent.com/SysBioChalmers/ecModels/chore/updateiML1515/eciML1515/model/eciML1515.xml"

model = cobra.io.read_sbml_model("iML1515.xml")
ecModel = cobra.io.read_sbml_model("eciML1515.xml")

## 2. Simulating models

### 2.1. Simulating the metabolic model

In [2]:
model.objective.expression.args[0]

1.0*BIOMASS_Ec_iML1515_core_75p37M

In [3]:
model.solver.timeout = 30
model.optimize()

,fluxes,reduced_costs
ALATA_D2,0.000000,-7.523353e-03
SHCHD2,0.000196,1.110223e-16
CPPPGO,0.000196,-2.168404e-19
GTHOr,0.217041,-5.561957e-17
DHORD5,0.000000,0.000000e+00
...,...,...
SUCCt1pp,0.000000,0.000000e+00
QUINDH,0.000000,-1.880838e-03
LCARSyi,0.000000,-1.880838e-03
BIOMASS_Ec_iML1515_core_75p37M,0.876997,-1.452221e-16


In [4]:
model.summary()

Note that only 4 metabolites are consumed: oxygen, glucose, ammonia and phosphate.

### 2.2. Simulating the enzyme-constrained model _as-is_

In [5]:
ecModel.objective.expression.args[0]

1.0*BIOMASS_Ec_iML1515_core_75p37M

In [6]:
ecModel.solver.timeout = 30
ecModel.optimize()

OptimizationError: solver status is 'unbounded'

### 2.3 Trying to fix the enzyme-constrained model

cobrapy cannot handle upper bounds = `Inf`, therefore we need to replace them with `1000` (standard in the field):

In [7]:
import math

ex_ubound = 0
for exchange in ecModel.exchanges:
    if math.isinf(exchange.upper_bound) or math.isinf(exchange.lower_bound):
        ex_ubound += 1
        
rr_ubound = 0
for reaction in ecModel.reactions:
    if math.isinf(reaction.upper_bound) or math.isinf(reaction.lower_bound):
        rr_ubound += 1
        

print(f'Unbound reactions -> {rr_ubound}\n Unbound exchanges -> {ex_ubound}')

Unbound reactions -> 6083
 Unbound exchanges -> 662


In [8]:
for reaction in ecModel.reactions:
    if reaction in model.reactions:
        reaction.bounds = model.reactions.get_by_id(reaction.id).bounds
    elif math.isinf(reaction.upper_bound):
        reaction.upper_bound = 1000

Pretty high objective function -> we need to lower the upper bound of glucose uptake:

In [16]:
ecModel.reactions.prot_O32583_exchange

Reaction identifier,prot_O32583_exchange
Name,prot_O32583_exchange
Memory address,0x07f168c652160
Stoichiometry,--> prot_O32583__91__c__93__ --> prot_O32583 [cytosol]
GPR,b4407
Lower bound,0.0
Upper bound,1000


In [9]:
ecModel.optimize()

,fluxes,reduced_costs
EX_acgam_e,0.0,0.0
EX_cellb_e,0.0,0.0
EX_chol_e,0.0,0.0
EX_pi_e,1000.0,0.0
EX_h_e,1000.0,0.0
...,...,...
prot_Q59385_exchange,0.0,-0.0
prot_Q6BEX0_exchange,0.0,-0.0
prot_Q6BF16_exchange,0.0,-0.0
prot_Q6BF17_exchange,0.0,-0.0


Nothing changed -> let's look at the summary of in/out fluxes:

In [ ]:
ecModel.summary()

We see that there are many uptake fluxes fully unconstrained. Let's fix all of them to zero except for the original 4 (oxygen, glucose, ammonia and phosphate):

In [ ]:
for reaction in ecModel.reactions:
    if len(reaction.metabolites) == 1 and reaction.name.endswith(" (reversible)"):
        reaction.lower_bound = 0
        reaction.upper_bound = 0

ecModel.reactions.EX_glc__D_e_REV.upper_bound = 10
ecModel.reactions.EX_o2_e_REV.upper_bound = 1000
ecModel.reactions.EX_nh4_e_REV.upper_bound = 1000
ecModel.reactions.EX_pi_e_REV.upper_bound = 1000

ecModel.optimize()

## 3. Cleanup

In [ ]:
import os
os.remove("iML1515.xml")
os.remove("eciML1515.xml")